In [0]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import sys
import os

In [0]:
spark = SparkSession.builder.getOrCreate()

In [0]:
all_sub_file_list  = os.listdir('/dbfs/mnt/lsde/group24/submissions')
all_sub_file_list.remove('RS_2020-04')
all_sub_file_list.remove('RS_2020-05')
all_sub_file_list.remove('RS_2020-08')
print(len(all_sub_file_list))

34


In [0]:
#last three years 2019.8~2022.8, top popular subreddit
#remove 2020.4, 2020.5, 2020.8
#because the comments JSON files of 2020.4, 2020.8, 2020.5 can not be read successfully
delete_list = ['t5_23wi2r', 't5_23n1uk', 't5_23msy5', 't5_23wj65'] 
file_path = 'dbfs:/mnt/lsde/group24/submissions/'
def select_subreddit():
    schema_sub = "subreddit string, subreddit_id string, num_comments long, score long"
    result = spark.createDataFrame(spark.sparkContext.emptyRDD(), schema_sub)
    for sub_file in all_sub_file_list:
        sub = spark.read.parquet(file_path + sub_file) \
                        .select(col('subreddit'),col('subreddit_id'),col('num_comments'),col('score'))
        result = result.union(sub)
    print('union finished')
    result = result.groupBy('subreddit','subreddit_id') \
             .agg(sum('num_comments').alias('total_num_comments'), sum('score').alias('total_score')) \
             .orderBy(desc('total_num_comments'),desc('total_score'))
    result = result.filter(col('subreddit_id').isin(delete_list) == False)
    print('rank finished')
    return result.limit(1000)
subreddit_rank_2 = select_subreddit()

union finished
rank finished


In [0]:
sub_id_list_2 = subreddit_rank_2.select(col('subreddit_id')).orderBy(asc('subreddit_id')).rdd.map(lambda r: r.subreddit_id).collect()

In [0]:
rank = spark.read.parquet('dbfs:/mnt/lsde/group24/subreddit_rank_2')
sub_id_list = rank.select(col('subreddit_id')).orderBy(asc('subreddit_id')).rdd.map(lambda r: r.subreddit_id).collect()

In [0]:
print(set(sub_id_list_2)^set(sub_id_list))

{'t5_35g9w', 't5_23msy5', 't5_2qhbe', 't5_2r40o', 't5_23wi2r', 't5_23wj65', 't5_23n1uk', 't5_2r0bp'}


In [0]:
display(subreddit_rank_2)

subreddit,subreddit_id,total_num_comments,total_score
AskReddit,t5_2qh1i,210748996,252785027
memes,t5_2qjpg,66391491,4646807245
politics,t5_2cneq,59743063,744031557
teenagers,t5_2rjli,57573011,437396890
wallstreetbets,t5_2th52,51278307,195978489
AmItheAsshole,t5_2xhvq,50482061,217504842
FreeKarma4U,t5_3hapm,40238800,23347270
worldnews,t5_2qh13,33161451,357604329
nfl,t5_2qmg3,31096963,96109448
nba,t5_2qo4s,30928961,136591400


In [0]:
write_path = 'dbfs:/mnt/lsde/group24'
subreddit_rank_2.write.format('parquet').mode("overwrite").save(write_path + "/subreddit_rank_2")

In [0]:
subreddit_rank_2.coalesce(1).write.mode("overwrite") \
    .option("mapreduce.fileoutputcommitter.marksuccessfuljobs","false") \
    .option("header","true") \
    .option("delimiter", "|") \
    .csv('dbfs:/FileStore/Group24/subreddit_rank_2.csv')

In [0]:
#The first time code
def rank_subreddit(year, month):
    write_path = '/mnt/lsde/group24'
    if year == 2020 and month in [4,5,8]
    if month < 10:
        file_path = '/mnt/lsde/group24/submissions/RS_' + str(year)+ '-0' + str(month)
    else:
        file_path = '/mnt/lsde/group24/submissions/RS_' + str(year)+ '-' + str(month)
    if not os.path.exists('/dbfs' + file_path):
        print('This file does not exist')
    else:
        submissions = spark.read.parquet('dbfs:' + file_path)
        #              .select(col('subreddit'), col('subreddit_id'), col('num_comments'),col('score'))
        subr_rank = submissions.groupBy('subreddit', 'subreddit_id') \
                            .agg(sum('num_comments').alias('num_comments'), sum('score').alias('score')) \
                            .orderBy(desc('num_comments'),desc('score'))
        print('dbfs:' + file_path+" has finished.")
        return subr_rank.limit(2000)

In [0]:
#####
def union_all(subr_list):
    result = spark.createDataFrame(spark.sparkContext.emptyRDD(), schema)
    for subr in subr_list:
        result = result.union(subr)
    result = result.groupBy('subreddit','subreddit_id') \
             .agg(sum('num_comments').alias('num_comments'), sum('score').alias('score')) \
             .orderBy(desc('num_comments'),desc('score'))
    return result.limit(1000)

In [0]:
#####
#last three years 2019.8~2022.8, top popular subreddit
subr_list = []
year_s =2019
year_e = 2022
month_s = 1
month_e = 12
for year in range(year_s,year_e + 1):
    if year == 2019:
        month_s = 8
    else:
        month_s = 1
    if year == 2022:
        month_e = 8
    else:
        month_e = 12
    for month in range(month_s,month_e + 1):
        subr = rank_subreddit(year, month)
        subr_list.append(subr)
write_path = 'dbfs:/mnt/lsde/group24'
subreddit_rank = union_all(subr_list)
subreddit_rank.write.format("parquet").mode("overwrite").save(write_path + "/subreddit_rank")
#subr_list[0].show()